http://www2.cao.ie/points/l8.php

***

In [1]:
#convenient http requests
import requests as rq

# regular expressions
import re

#dates and times
import datetime as dt

# for dataframes import pandas
# v useful to get data from spreadsheets
import pandas as pd

#for downloading files to python
import urllib.request as urlrq



In [2]:
now = dt.datetime.now()

In [3]:
#datetime function has a data structure called datetime
#field for time, teat ,month etc
#dt.datetime.now()
now = dt.datetime.now()

In [4]:
# format now as a string
#if have datetimestamp as separate and unique at the stasrt of the notebook
# then potentially can overwrite but on positive side all files 
# created in logbook run will ahve the same datetimestamp sp neater and can track
#2020 file is equivalent to 2021c etc
now.strftime('%Y%m%d_%H%M%S')

'20211109_211049'

In [5]:
print(now)

2021-11-09 21:10:49.113282


# 2021 points

http://www2.cao.ie/points/l8.php

***


In [6]:
# fetch the CAO points URL
# resp= response from the URL
# rq= request library
# get= get the url (http version)
resp = rq.get ('http://www2.cao.ie/points/l8.php')
#have a quick peek
resp
# resp [200] means html response is in order.  [404] if returns means it is not found .  These are http codes

<Response [200]>

### use the current time as a filename - need to remove colons, decimal points etc

In [7]:
# good programmers use only alphanumeric characters ,dash and dot in file names
#create a filepath for the original data
#use year at start to ensure alphabetic sorted when filed
# below creates the csv file and saves it in a new 'data' folder in active location  ie. \repo\Fun_of..
pathhtml = 'data/cao2021_' + now.strftime('%Y%m%d_%H%M%S')  + '.html'

### the server uses the wrong encoding fix it before saving

In [8]:
#RESP = RESPONSE
#'resp.text' takes the response and converts it to text- but still has original errors
#thus not best way to save THIS original file
#but resp.text is a typical way to save

In [9]:
original_encoding = resp.encoding
resp.encoding = 'cp1252'

In [10]:
#save the original html file

with open(pathhtml, 'w') as f:
    f.write(resp.text)    

In [11]:
#resp.text
#\n is a new line (common Unix command)

In [12]:
#how to loop throught the linesv in request. iterate through the lines in request
#for line in resp.iter_lines():
   # print(line)

In [13]:
#compile the regular expression for matching lines
# re= regular expression. Each character is a string but together form an instruction.
# r. = 'raw' used in python. It means don't evaluate backslashes in encountered string literals  i.e. \n \t i.e 
re_course = re.compile(r'([A-Z]{2}[0-9]{3})  (.*)([0-9]{3})(\*?) *')

#shortened RE to simplify i.e. does the line have two alphabetic characters followed by three numbers followed by anything else
#re_course = re.compile(r'([A-Z]{2}[0-9]{3})(.*)')

<br>

**error on the server= way to fix**

match only the lines we want -the ones representing courses 
 technically the server says we should decode as:
 
content-type: text/html; charset: iso-8859-1

  but
    -one line uses /x96 which isn't defined in iso-8859-1
    -therefore we used the similiar decoding standard (codec) cp-1252 which is very similar but includes \x96

### loop through the lines of the response
***

In [14]:
#the filepath for the csv file- which is the output of the below script
path2021 = 'data/cao2021_csv' + now.strftime('%Y%m%d_%H%M%S')  + '.csv'

In [15]:
#for the below if used in assessment should be cleaned up. Check regular expressions are working properly
# if want to save output of this then need a filepath and file format like the original data above

no_lines = 0
#open the csv file for writing
with open(path2021, 'w') as f:
    # write a header row
    f.write(','.join(['code', 'title', 'pointsR1', 'pointsR2']) + '\n')
   # f.write(resp.text)   
    #loop through the lines of the response content
    for line in resp.iter_lines():
        #decode the line using the encoding not flagged by the response from CAO
        dline = line.decode('cp1252')
        #match only the lines representing courses
        if re_course.fullmatch(dline):
            #add one to the line counter
            no_lines = no_lines + 1
            #print(line) no need to print so commented out
            # version 1 using regular expression
            # csv_version = re_course.sub(r'\1,\2,\3,\4', dline) version 1 using re commented out
            #print(csv_version)  no need to print so commented out
            #alternate way than regular expression is to use python in built split function
            # split splits the line
            #(+ ) instructs to do this where two or more spaces- NB the spaces represented before the '+' below are v important
            linesplit = re.split('  +', dline)
      
            f.write(','.join(linesplit) + '\n') 
print(f"total number of lines is {no_lines}.")       

total number of lines is 922.


<br>

#### helper function

In [16]:
# takes in s (str) - checks is 1st character #? 
#if yes then stores in portfolio a # otherwise it stores an empty string
def points_to_array(s):
    portfolio = ''
    if s[0] == '#':
        portfolio = '#'
        random = ''
        #checks the last character  if its anasterisk then it stores 
        if s[-1] == '*':
            random = '*'
            # loops through each characters in string for i in s
            # + is string concatenation 
            # + is not addition
        points = ''
        for i in s:
            if i.isdigit():
                points = points + 1
                return [points, portfolio, random]
            
# 'would recommend changing course _points[0], course_points[1]] into arrays with three things in them
#hash character , empty string, points ,random has- maybe don't need if read in pandas'

#### Extra lines in html- how to reconcile the # lines of html vs the csv
- i.e. total number of lines output in jupyter code
- open VSC
- delete asll the lines that don't want
- delete preamble & sections within colleges

#find \n\n and replace with \n
in RE turned on : ^ looks for start of line $ looks for end of line# cant find and replace with these but can find S&F of lines


#### alternate way to sort
- different to RE but similar result
- more manual


In [17]:
 no_lines = 0
#open the csv file for writing
with open(path2021, 'w') as f:
    # write a header row
    f.write(','.join(['code', 'title', 'pointsR1', 'pointsR2']) + '\n')
    #debug:
            ##print(len(linesplit), linesplit ,dline)
            #this will show you how the RE is splitting 
            #i.e. expect '4' for len(length) of linesplit, then in square brackets how it splits 'xxx', then outside square brackets the string- 'dline'
            
            #print out just the course code # :5b means simply print out the characters up to position 5 (0-4 in python)
            #course_code = dline[:5]
            #print (course_code)
            
            #print out just the course title- starts at 7n in..trial and error re run until get longest course title
            #course_title = dline[7:57]
            #print (course_title)     
            
            #round one points - prints out both round 1 and round 2  Note: can't separate 'round' and '1'
            #course_round1 = dline[60:]
            #print (course_round1)
            
            #can edit the RE and re-run these to debug further
            
            #further debugging
            #split on the space. Has a look for one ore more space and splits it
            #course_points = re.split('  +' , dline[60:])
            #print(f"'{course_code} {course_points}'")
            
            #last line is returning anything but two substrings back- create an exception
           # if len(course_points) != 2:
                #course_points = course_points[:2]
               # join the fields usign a comma
               # linesplit = [course_code,course_title, course _points[0], course_points[1]]
                
            #print(','.join(linesplit))
            #f.write(csv_version + '\n') - saving of old RE way
            #rejoin the substrings with commas in-between
            #f.write(','.join(linesplit) + '\n')
            
            #result - output is a csv file!
    for line in resp.iter_lines():
        dline = line.decode('cp1252')
        if re_course.fullmatch(dline):
            no_lines = no_lines + 1
            course_code = dline[:5]
            course_title = dline[7:57].strip()
            course_points = re.split('  +' , dline[60:])
            if len(course_points) != 2:
                course_points = course_points[:2]
            linesplit = [course_code,course_title, course_points[0], course_points[1]]
            f.write(','.join(linesplit) + '\n') 
print(f"total number of lines is {no_lines}.")  
# output path2021 is a csv file
#note pandas coud convert html to csv but better to use own code and save as csv- use pandas thereafter

total number of lines is 922.


stripstrings = ' abc     ' <br>
**s.strip()** strips the white space** <br>

<br>

**NB** it was verfied as pof  xx yy zz that there were  949 courses exactly in the CAO 2021 points list

In [18]:
#old code
#how to loop throught the linesv in request. iterate through the lines in request using a regular expression
#for line in resp.iter_lines():
   # if re.match('[A-Z]{2}[0-9]{3}  .* *(0-9){3} *', line.decode('utf-8') ):
               # print(line)

In [19]:
df2021 = pd.read_csv(path2021, encoding='cp1252')

<br>

# 2020 points

http://www.cao.ie/index.php?page=points&p=2020

***

### download and parse the excel spreadsheet using pandas

In [20]:
#create a variable for the url to keep it neat
url2020 = 'http://www2.cao.ie/points/CAOPointsCharts2020.xlsx'

In [21]:
#download and parse the excel spreadsheet
#skip the 1st 10 preamble rows
df2020 = pd.read_excel (url2020, skiprows = 10)
# however it reads the 1st 10 preamble headings 
# could save file and manually delete 1st 10 rows but better to use pandas functionality

In [22]:
df2020

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Manufacturing and processing,Manufacturing Engineering,WD208,188,NaN,NaN,NaN,188,NaN,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
#spot check of data
# use loc or iloc functionality to check data
df2020.iloc[12]
#check the final line in the spreadsheet
# reminder 1st 11 rows are heading #excel identifies 1st row as zero
df2020.iloc[753]
# can also count backwards to last row
df2020.iloc[-1]

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Mechanical and Manufacturing Engineering
COURSE CODE2                                                           WD230
R1 POINTS                                                                253
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      253
EOS Random *                                                             NaN
EOS Mid-point                                                            369
LEVEL                                                                      8
HEI                                        Waterford Institute of Technology
Test/Interview #                                                         NaN

<br>

### save original file 

***

In [24]:
pathxlsx = 'data/cao2020_' + now.strftime('%Y%m%d_%H%M%S')  + '.xlsx'
now.strftime('%Y%m%d_%H%M%S')
#save the original html file
with open(pathxlsx, 'w') as f:
    f.write(resp.text)   

In [25]:
#urllib save the original file to disk

urlrq.urlretrieve (url2020, pathxlsx)

('data/cao2020_20211109_211049.xlsx',
 <http.client.HTTPMessage at 0x2b4cb826070>)

<br>

#### create a filepath for pandas data
***


In [26]:
#save the output to a csv file
path2020 = 'data/cao2020_' + now.strftime('%Y%m%d_%H%M%S')  + '.csv'
df2020.to_csv(path2020)

<br>

https://pandas.pydata.org/docs/reference/api/pandas.read_excel.htmlpandas.read_excel <br> function signature in pandas excel webpage: (io, sheet_name=0, header=0, names=None, index_col=None, usecols=None, squeeze=False, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skiprows=None, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, parse_dates=False, date_parser=None, thousands=None, comment=None, skipfooter=0, convert_float=None, mangle_dupe_cols=True, storage_options=None)¶
skiprows is what we need

## fetch earlier year points data and save in data files

<br>

## 2019 points
-as pdf option here is to read do it manually before getting pandas to read it as a csv

http://www.cao.ie/index.php?page=points&p=2019

***

-Note the 2019 and earlier points are in pdf format<br>
Main points inndex url: <br>
https://www.cao.ie/index.php?page=points


<br>
### steps to reproduce the 2019 editable:
****


Manual way of extracting:
1. save pdf in repository
2. open original with MS Word (may need to 'choose other app to open' in dropdown
3. MS Word converts pdf to Word
4. Save file as xxxxx.docx format
5. Resave a copy to edit in repo as xxxxx_edited.docx
6  Delete headers and footers
7. delete preamble on page 1
8. Ctrl 'A', Ctrl 'C' to copy all data in the 21 separate tables
9. Open notepad++ and paste 'Ctrl V'


### manual editing in notepad ++
1. Add HEI to header
2.Select the 'College/Institute' i.e. 'Athlone Institute of Technology'
3. Ctrl X to cut
4. Move cursor to in front of AIT 1st course line
5. Hold down ALT key and drag cursor down to all AIT courses
6. Ctrl V to paste - click 'tab' key to create tabs
7. Delete blank lines
8. Put double quotes around college names as they have commas [select start of string with cursor-alt-type "]
9. Change backticks to apostrophes
10. optional : \t  in extended file 'replace all' with comma: replaces all the tabs with commas <br> however not recommended here as commas elsewhere will confuse. <br> best to leave tabs as delimiters  e.g. in effect its as 'tsv' tab serapated file but can save as csv
11. find \\t replace with \t finds the double tabs and removes them
11. save file

In [27]:
df2019 = pd.read_csv('data/cao2019_20211102_191031_edited.csv', sep ='\t') 

In [28]:
df2019

,HEI,Course Code,INSTITUTION and COURSE,EOS,Mid
0,,NaN,NaN,NaN,NaN
1,Athlone Institute of Technology,AL801,Software Design with Virtual Reality and Gaming,304,328.0
2,Athlone Institute of Technology,AL802,Software Design with Cloud Computing,301,306.0
3,Athlone Institute of Technology,AL803,Software Design with Mobile Apps and Connected...,309,337.0
4,Athlone Institute of Technology,AL805,Network Management and Cloud Infrastructure,329,442.0
...,...,...,...,...,...
974,Waterford Institute of Technology,WD200,Arts (options),221,296.0
975,Waterford Institute of Technology,WD210,Software Systems Development,271,329.0
976,Waterford Institute of Technology,WD211,Creative Computing,275,322.0
977,Waterford Institute of Technology,WD212,Recreation and Sport Management,274,311.0


In [29]:
# fetch the CAO points URL.  The url for earlier years takes you to a pdf version
# However the python code to fetch appears to work
resp = rq.get ('http://www2.cao.ie/points/lvl8_19.pdf')
#have a quick peek
resp
# resp [200] means html response is in order.  [404] if returns means it is not found .  These are http codes

<Response [200]>

In [30]:
path2019 = 'data/cao2019_' + now.strftime('%Y%m%d_%H%M%S')  + '.html'
now.strftime('%Y%m%d_%H%M%S')
#save the original html file
with open(path2019, 'w') as f:
    f.write(resp.text)   

UnicodeEncodeError: 'charmap' codec can't encode characters in position 12-15: character maps to <undefined>

#### fetch 2018

**propose to go through the same process as 2019**

In [31]:
resp = rq.get ('http://www2.cao.ie/points/lvl8_18.pdf')
resp

<Response [200]>

In [33]:
path2018 = 'data/cao2018_' + now.strftime('%Y%m%d_%H%M%S')  + '.html'
now.strftime('%Y%m%d_%H%M%S')
#save the original html file
with open(path2018, 'w') as f:
    f.write(resp.text) 

UnicodeEncodeError: 'charmap' codec can't encode characters in position 12-15: character maps to <undefined>

###How to convert pdf to csv in python- further reading: <br>
-https://stackoverflow.com/questions/49560486/how-to-convert-pdf-to-csv-with-tabula-py

<br>

### Concatenate and Merge dataframes

how to concatenate with pandas <br>
note 'ignore_index'= True <br>
https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html <br>
how to manage duplicates in a dataframe <br>
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html <br>
how to return uniques in dastaframe columns <br>
https://pandas.pydata.org/docs/reference/api/pandas.Series.unique.html <br>
how to drop duplicates in dataframe <br>
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html

In [34]:
#extract the required columns (course and points) from the dataframe
courses2021= df2021[['code' , 'title']]

In [35]:
courses2021

,code,title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
917,WD211,Creative Computing
918,WD212,Recreation and Sport Management
919,WD230,Mechanical and Manufacturing Engineering
920,WD231,Early Childhood Care and Education


In [36]:
courses2020= df2020[['COURSE CODE2' , 'COURSE TITLE']]
# change the title of the columns to match 2021
courses2020.columns = ['code' ,'title']

In [37]:
courses2020

,code,title
0,AC120,International Business
1,AC137,Liberal Arts
2,AD101,"First Year Art & Design (Common Entry,portfolio)"
3,AD102,Graphic Design and Moving Image Design (portfo...
4,AD103,Textile & Surface Design and Jewellery & Objec...
...,...,...
1459,WD208,Manufacturing Engineering
1460,WD210,Software Systems Development
1461,WD211,Creative Computing
1462,WD212,Recreation and Sport Management


In [38]:
#merge or concatenate the two datasets into one dataframe
#set ignore_index = True (default is False)
allcourses = pd.concat([courses2021, courses2020], ignore_index=True)

In [39]:
allcourses

,code,title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
2381,WD208,Manufacturing Engineering
2382,WD210,Software Systems Development
2383,WD211,Creative Computing
2384,WD212,Recreation and Sport Management


In [40]:
# returns a false for every row that are not duplicated
allcourses.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
2381    False
2382     True
2383     True
2384     True
2385     True
Length: 2386, dtype: bool

In [41]:
#finds all extra copies of duplicated rows
#duplicated by default doesnt set the 1st iteration of the matching rows as true (returns false)
allcourses[allcourses.duplicated()] 

,code,title
922,AC120,International Business
923,AC137,Liberal Arts
925,AD102,Graphic Design and Moving Image Design (portfo...
928,AD204,Fine Art (portfolio)
929,AD211,Fashion Design (portfolio)
...,...,...
2377,WD200,Arts (options)
2382,WD210,Software Systems Development
2383,WD211,Creative Computing
2384,WD212,Recreation and Sport Management


In [42]:
allcourses[allcourses.duplicated(keep = False)]

,code,title
4,AL810,Quantity Surveying
5,AL820,Mechanical and Polymer Engineering
6,AL830,General Nursing
9,AL836,Nutrition and Health Science
10,AL837,Sports Science with Exercise Physiology
...,...,...
2377,WD200,Arts (options)
2382,WD210,Software Systems Development
2383,WD211,Creative Computing
2384,WD212,Recreation and Sport Management


In [43]:
#returns back the dataframe with all duplicates dropped
allcourses.drop_duplicates()

,code,title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
2376,WD195,Architectural and Building Information Modelli...
2378,WD205,Molecular Biology with Biopharmaceutical Science
2379,WD206,Electronic Engineering
2380,WD207,Mechanical Engineering


In [44]:
# just look for duplicates of the code- as the course titles are likely different from year to year
allcourses[allcourses.duplicated(subset =['code'])]

,code,title
922,AC120,International Business
923,AC137,Liberal Arts
924,AD101,"First Year Art & Design (Common Entry,portfolio)"
925,AD102,Graphic Design and Moving Image Design (portfo...
926,AD103,Textile & Surface Design and Jewellery & Objec...
...,...,...
2377,WD200,Arts (options)
2382,WD210,Software Systems Development
2383,WD211,Creative Computing
2384,WD212,Recreation and Sport Management


In [45]:
#drop duplicates based on the same code
allcourses.drop_duplicates(subset=['code'])

,code,title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
2371,WD188,Applied Health Care
2378,WD205,Molecular Biology with Biopharmaceutical Science
2379,WD206,Electronic Engineering
2380,WD207,Mechanical Engineering


In [46]:
#drop duplicates based on the same code
#creates a new dataframe as by default-this is identified by 'inplace=False'
#ignore_index cleans up the duplicates in the index i.e. counts just one of duplicates indexes
allcourses.drop_duplicates(subset=['code'], inplace=True, ignore_index=True)

In [47]:
#as we used inplace=True then we have permanentlty unduplicated the dataframe for codes
#when run allcourses this time the default dtaframe has no course code duplicates
allcourses

,code,title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
1508,WD188,Applied Health Care
1509,WD205,Molecular Biology with Biopharmaceutical Science
1510,WD206,Electronic Engineering
1511,WD207,Mechanical Engineering


reset index in pandas dataframe<br>
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html

In [48]:
#sort by courses
allcourses.sort_values ('code')

,code,title
171,AC120,International Business
172,AC137,Liberal Arts
302,AD101,First Year Art and Design (Common Entry portfo...
303,AD102,Graphic Design and Moving Image Design (portfo...
304,AD103,Textile and Surface Design and Jewellery and O...
...,...,...
917,WD211,Creative Computing
918,WD212,Recreation and Sport Management
919,WD230,Mechanical and Manufacturing Engineering
920,WD231,Early Childhood Care and Education


In [49]:
allcourses.loc[0]

code                                              AL801
title    Software Design for Virtual Reality and Gaming
Name: 0, dtype: object

In [50]:
allcourses.iloc[0]

code                                              AL801
title    Software Design for Virtual Reality and Gaming
Name: 0, dtype: object

<br>

## Join to the Points 
***
<br>


worth knowing: pandas dataframe.join
how:
**main event is left**
- left join: - whatever df is on the left (join call) the other is on the right . keeps everything from the left
- only join: keeps values in BOTH dataframes
- right join: keeps everything in the right dataframe
- full-outer join merges everything- (have a lot of NaN's)

In [51]:
#set the index to the code column
df2021.set_index ('code', inplace=True)
df2021.columns = ['title' , 'points_r1_2021', 'points_r2_2021']
df2021

,title,points_r1_2021,points_r2_2021
code,,,
AL801,Software Design for Virtual Reality and Gaming,300,NaN
AL802,Software Design in Artificial Intelligence for...,313,NaN
AL803,Software Design for Mobile Apps and Connected ...,350,NaN
AL805,Computer Engineering for Network Infrastructure,321,NaN
AL810,Quantity Surveying,328,NaN
...,...,...,...
WD211,Creative Computing,270,NaN
WD212,Recreation and Sport Management,262,NaN
WD230,Mechanical and Manufacturing Engineering,230,230


In [52]:
#pull in the points from 2021 and 2020
#see pandas concatenate reference above

In [53]:
#set the column as the index
allcourses.set_index('code', inplace =True)

In [56]:
#just merge what you need from the 2021 dataframe i.e. code and points
#where NaN it defines as code that was not run in 2021
allcourses = allcourses.join(df2021[['points_r1_2021']])
allcourses

,title,points_r1_2021
code,,
AL801,Software Design for Virtual Reality and Gaming,300
AL802,Software Design in Artificial Intelligence for...,313
AL803,Software Design for Mobile Apps and Connected ...,350
AL805,Computer Engineering for Network Infrastructure,321
AL810,Quantity Surveying,328
...,...,...
WD188,Applied Health Care,NaN
WD205,Molecular Biology with Biopharmaceutical Science,NaN
WD206,Electronic Engineering,NaN


In [57]:
# just need to join the pointd from 2020
df2020_r1= df2020 [['COURSE CODE2', 'R1 POINTS']]
df2020_r1.columns = ['code', 'points_R1_2020']
df2020_r1

,code,points_R1_2020
0,AC120,209
1,AC137,252
2,AD101,#+matric
3,AD102,#+matric
4,AD103,#+matric
...,...,...
1459,WD208,188
1460,WD210,279
1461,WD211,271
1462,WD212,270


In [58]:
# set the index to the code column
df2020_r1.set_index ('code', inplace=True)
df2020_r1                 

,points_R1_2020
code,
AC120,209
AC137,252
AD101,#+matric
AD102,#+matric
AD103,#+matric
...,...
WD208,188
WD210,279
WD211,271


In [61]:
#just merge what you need from the 2021 dataframe i.e. code and points
#where NaN it defines as code that was not run in 2021
#join2020 points to all courses
#allcourses = allcourses.join(df2021[['pointsR1']])
#allcourses

In [60]:
# join 2020 points to all courses
allcourses = allcourses.join(df2020_r1)
allcourses

,title,points_r1_2021,points_R1_2020
code,,,
AL801,Software Design for Virtual Reality and Gaming,300,303
AL802,Software Design in Artificial Intelligence for...,313,332
AL803,Software Design for Mobile Apps and Connected ...,350,337
AL805,Computer Engineering for Network Infrastructure,321,333
AL810,Quantity Surveying,328,319
...,...,...,...
WD188,Applied Health Care,NaN,201
WD205,Molecular Biology with Biopharmaceutical Science,NaN,228
WD206,Electronic Engineering,NaN,179


https://www.prepressure.com/library/technology/ascii-binary-hex

http://www.i18nqa.com/debug/table-iso8859-1-vs-windows-1252.html

https://docs.python.org/3/library/datetime.html

In [ ]:
#strftime turns datetime to strings
#strptime turns strings into datetime